In [23]:
import fastbook
fastbook.setup_book()

C:\Python399\lib\site-packages\fastbook\__init__.py:19: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [24]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [176]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

def linear1(xb): return xb@weights + bias

def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()


In [140]:
path = untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')


In [164]:
train_path = (path/'training').ls().sorted() # Read the folder training and list all subfolders: 0 to 9
x_train = []
y_train = []
for i, pat in enumerate(train_path): # Go through each subfolder of 'training'
    num = [tensor(Image.open(o)).float() for o in (pat).ls()] # Open each image of the respective subfolder as float matrix (28x28) and store it in the x_train vector
    x_train += num
    y_train += tensor([i]*len(num)).unsqueeze(1) # y_train is simply the respective number as label (0 to 9)
x_train = torch.cat([x.view(-1,28*28) for x in x_train]) # Convert the rank-3 vector to rank-2
y_train = torch.stack(y_train)
training_dataset = list(zip(x_train,y_train))

In [165]:
test_path = (path/'testing').ls().sorted() # Read the folder 'testing' and list all subfolders: 0 to 9
x_test = []
y_test = []
for i, pat in enumerate(test_path): # Go through each subfolder of 'testing'
    num = [tensor(Image.open(o)).float() for o in (pat).ls()] # Open each image of the respective subfolder as float matrix (28x28) and store it in the x_testing vector
    x_test += num
    y_test += tensor([i]*len(num)).unsqueeze(1) # y_train is simply the respective number as label (0 to 9)
x_test = torch.cat([x.view(-1,28*28) for x in x_test]) # Convert the rank-3 vector to rank-2
y_test = torch.stack(y_test)
testing_dataset = list(zip(x_test,y_test))

In [166]:
training_dl = DataLoader(training_dataset,batch_size=256)

In [167]:
valid_dl = DataLoader(testing_dataset,batch_size=256)

In [168]:
weights = init_params((28*28,1))
bias = init_params(1)

In [174]:
batch = x_train[:1000]
batch.shape

torch.Size([1000, 784])

In [177]:
calc_grad(batch, y_train[:1000], linear1)
weights.grad.mean(),bias.grad

(tensor(0.0715), tensor([0.0012]))

In [184]:
def train_epoch(model):
    for xb,yb in training_dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()

In [185]:
def train_model(model, epochs):
    for i in range(epochs):
        train_epoch(model)
        # print(validate_epoch(model), end=' ')

In [186]:
lr =1.

linear_model = nn.Linear(28*28,1) # This method is native of fastai, it simply does batch@weights + bias
opt = SGD(linear_model.parameters(),lr) # Optimizer, SGD is a module native of fastai
train_model(linear_model,20)

In [187]:
dls = DataLoaders(training_dl,valid_dl)

In [ ]:
# Check nn.Sequential
# Simple net

In [ ]:
learn = Learner(dls, nn.Linear(28*28,1), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [27]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

In [28]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]

In [29]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255


In [30]:
valid_3_tens = torch.stack([tensor(Image.open(o)) 
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o)) 
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255

In [31]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)


In [30]:
dset = list(zip(train_x,train_y))

In [36]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [35]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [36]:
weights = init_params((28*28,1))
bias = init_params(1)

In [37]:
def linear1(xb): return xb@weights + bias
preds = linear1(train_x)
preds

tensor([[-3.8565],
        [-5.0067],
        [-9.2098],
        ...,
        [-8.2678],
        [ 0.1213],
        [ 1.4608]], grad_fn=<AddBackward0>)

In [38]:
corrects = (preds>0.5).float() == train_y
corrects

tensor([[False],
        [False],
        [False],
        ...,
        [ True],
        [ True],
        [False]])